<a href="https://colab.research.google.com/github/gabrielliman/Ising-Model/blob/main/IFC_IsingModel-cp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numba import jit, njit, config, __version__, errors
import numpy as np
import random
import matplotlib.pyplot as plt

In [ ]:
@jit(nopython=True)
def vizinhos(N):
  #Define a tabela de vizinhos
  L=int(np.sqrt(N))
  viz = np.zeros((N,4),dtype=np.int16)
  for k in range(N):
    viz[k,0]=k+1
    if (k+1) % L == 0: viz[k,0] = k+1-L
    viz[k,1] = k+L
    if k > (N-L-1): viz[k,1] = k+L-N
    viz[k,2] = k-1
    if (k % L == 0): viz[k,2] = k+L-1
    viz[k,3] = k-L
    if k < L: viz[k,3] = k+N-L
  return viz

In [ ]:
@jit(nopython=True)
def calcular_energia(s,viz):
  #Calcula a energia da configuração representada no array s
  N=len(s)
  ener = 0
  for i in range(N):
    h = s[viz[i,0]]+s[viz[i,1]] # soma do valor dos spins a direita e acima
    ener -= s[i]*h
  return ener

In [ ]:
@jit(nopython=True)
def calcular_magnetizacao(s):
  return sum(s)

In [ ]:
@jit(nopython=True)
def expos(beta):
  ex = np.zeros(5,dtype=np.float32)
  ex[0]=np.exp(8.0*beta)
  ex[1]=np.exp(4.0*beta)
  ex[2]=1.0
  ex[3]=np.exp(-4.0*beta)
  ex[4]=np.exp(-8.0*beta)
  return ex

In [ ]:
def alg_metropolis(L, temp_inicial, n_passos = 1000):
  tam_rede = (L ** 2)  # Entre 24 ** 2 e 100 ** 2
  beta = 1/temp_inicial
  ex = expos(beta)
  s = [random.choice([-1, 1]) for _ in range(tam_rede)]
  viz = vizinhos(tam_rede)
  energias = np.zeros(n_passos)
  energia_atual = calcular_energia(s, viz)
  magnetizacoes = np.zeros(n_passos)
  magnetizacao_atual = calcular_magnetizacao(s)
  magnetizacao_atual = 0
  for iter in range(n_passos * tam_rede):
    i = random.randint(0, tam_rede-1)
    h = s[viz[i,0]]+s[viz[i,1]]+s[viz[i,2]]+s[viz[i,3]] # soma dos vizinhos
    de = int(s[i]*h*0.5+2)
    P = ex[de]
    r = random.uniform(0, 1)
    if (r <= P):
      energia_atual += 2 * s[i] * h
      magnetizacao_atual = magnetizacao_atual + (-1) * 2 * s[i]
      s[i] = -s[i]
    else:
      pass
    if (iter % tam_rede == 0):
      energias[int(iter/tam_rede)] = energia_atual
      magnetizacoes[int(iter/tam_rede)] = magnetizacao_atual
  return energias, magnetizacoes

In [ ]:
def fazer_alg_metropolis_n_vezes(n, L, temp_inicial, n_passos = 1000):
  arr_energias = np.zeros((n,n_passos))
  arr_magnetizacoes = np.zeros((n,n_passos))
  for i in range(n):
    arr_energias[i], arr_magnetizacoes[i] = alg_metropolis(L, temp_inicial, n_passos)
  return arr_energias, arr_magnetizacoes

In [ ]:
def plotar_grafo_linha(arr_dados, label_x, label_y, title = ""):
  x = range(len(arr_dados[0]))
  for y in arr_dados:
    plt.plot(x, y)

  # Add labels and a title
  plt.xlabel(label_x)
  plt.ylabel(label_y)
  plt.title(title)

  # Display the plot
  plt.show()


1) Gere uma configuração inicial para o sistema (aleatória, por exemplo).\
2) Escolha um dos spins da rede (𝑆").\
3) Determine a diferença de energia caso o spin 𝑆" fosse flipado, Δ𝐸.\
4) Calcule 𝑃 = 𝑒^(-beta * del_E), e compare com um número aleatório, 𝑟, uniformemente
distribuído no intervalo (0,1).\
    a. Se 𝑟 ≤ 𝑃, aceite a nova configuração, ou seja, flipe o spin fazendo 𝑆" =
−𝑆".\
    b. Se 𝑟 > 𝑃, mantenha o sistema na configuração em que ele se
encontrava.\
5) Volte ao passo 2.\


In [ ]:
L = 32
N = L**2
temp_inicial = 1.5
beta = 1 / temp_inicial
n_passos = 1000
n = 5
arr_energias, arr_magnetizacoes = fazer_alg_metropolis_n_vezes(n, L, temp_inicial, n_passos)

In [ ]:
print(arr_energias.shape)
print(len(arr_energias[0]))

In [ ]:
plotar_grafo_linha(arr_energias, "número de passos Monte Carlo", "energia", "Variação da energia")

In [ ]:
plotar_grafo_linha(arr_magnetizacoes, "número de passos Monte Carlo", "magnetização", "Variação da magnetização")

In [ ]:
L = 32
temp_inicial = 0.4
limit=3.0
step=0.2
n_passos = 1000
n = 5
for temp in range(int(temp_inicial*100),int(limit*100+1),int(step*100)):
    temp=temp/100
    if (temp==temp_inicial):
        arr_energias, arr_magnetizacoes = fazer_alg_metropolis_n_vezes(n, L, temp, n_passos)
    else:
        arr_energias_aux, arr_magnetizacoes_aux = fazer_alg_metropolis_n_vezes(n, L, temp, n_passos)
        arr_energias=np.concatenate((arr_energias,arr_energias_aux),0)
        arr_magnetizacoes=np.concatenate((arr_magnetizacoes,arr_magnetizacoes_aux),0)
arr_energias.shape()

In [ ]:
temp=temp_inicial
for i in range(0,arr_energias.shape[0],n):
    plotar_grafo_linha(arr_magnetizacoes[i:i+n], "número de passos Monte Carlo para temperatura = " + str(round(temp,2)), "magnetização", "Variação da magnetização")
    temp=temp+step

In [ ]:
temp=temp_inicial
for i in range(0,arr_energias.shape[0],n):
    plotar_grafo_linha(arr_energias[i:i+n], "número de passos Monte Carlo para temperatura = " + str(round(temp,2)), "energia", "Variação da energia")
    temp=temp+step

# PDF 2

In [ ]:
# Cálculo do Calor Específico
def calc_specific_heat(arr, beta, N):
  """
  Calculates the specific_heat of a system given its energy.
  ------------
  Parameters:
    - arr (np.ndarray): array of energies of the system.
    - beta (float): constant beta.
    - N (int): number of particles in the system.
  Returns:
    - specific_heat (float): specific heat of the system.
  """
  mean_energy = np.mean(arr_energias, axis=1)
  mean_energy_squared = np.mean(arr_energias**2, axis=1)
  specific_heat = (mean_energy_squared - mean_energy**2)*(beta**2)/N
  return specific_heat

In [ ]:
specific_heat = calc_specific_heat(arr_energias, beta, N)

In [ ]:
temp=temp_inicial
for i in range(0,arr_energias.shape[0],n):
  print("Calor específico para iterações com temperatura = " + str(round(temp,1)))
  for j in range(n):
    print(calc_specific_heat(arr_energias[i+j], beta, N))
  temp=temp+step

In [ ]:
# Cálculo do Erro Estatístico
from math import sqrt

def calc_stat_err(arr):
  """
  Calculates the statistical error of a measure.
  ------------
  Parameters:
    - arr (np.ndarray): array of measures from which the error will be calculated.
  Returns:
    - err (integer): the statistical error of the measure.
  """
  avg = arr.mean()
  sum = ((avg - arr)**2).sum()
  err = sqrt(sum / (len(arr) - 1))
  return err


In [ ]:
calc_stat_err(specific_heat)

In [ ]:
# Cálculo da Susceptibilidade Magnética
def calc_magnet_susc(arr, beta, N):
  """
  Calculates the magnetic susceptibility of a system given its magnetization.
  ------------
  Parameters:
    - arr (np.ndarray): array of magnetizations of the system.
    - beta (float): constant beta.
    - N (int): number of particles in the system.
  Returns:
    - magnet_susc(float): magnetic susceptibility of the system.
  """
  mean_energy = np.mean(arr_energias, axis=1)
  mean_energy_squared = np.mean(arr_energias**2, axis=1)
  magnet_susc = (mean_energy_squared - mean_energy**2)*beta/N
  return magnet_susc

In [ ]:
calc_magnet_susc(arr_magnetizacoes, beta, N)

In [ ]:
def vary_temp(L, temp_inicial, limit, step, n_passos, n):
  beta = 1/temp_inicial
  N = L**2

  for temp in range(int(temp_inicial*100),int(limit*100+1),int(step*100)):
    temp = temp/100
    if (temp==temp_inicial):
        arr_energias, arr_magnetizacoes = fazer_alg_metropolis_n_vezes(n, L, temp, n_passos)
    else:
        arr_energias_aux, arr_magnetizacoes_aux = fazer_alg_metropolis_n_vezes(n, L, temp, n_passos)
        arr_energias= np.concatenate((arr_energias,arr_energias_aux),0)
        arr_magnetizacoes=np.concatenate((arr_magnetizacoes,arr_magnetizacoes_aux),0)
  return arr_energias, arr_magnetizacoes



In [ ]:
def plot_vary_temp(temp_inicial, arr_energias, arr_magnetizacoes, type):
  match tipo:
    case 0: # Calor específico
      for i in range(0,arr_energias.shape[0],n):
        plotar_grafo_linha(calc_specific_heat(arr_energias[i:i+n], beta, N), "número de passos Monte Carlo para temperatura = " + str(round(temp,2)), "Calor Específico", "Variação do Calor Específico")
        temp= temp + step
    case 1: # Susceptibilidade magnética
      for i in range(0,arr_energias.shape[0],n):
        plotar_grafo_linha(calc_magnet_susc(arr_magnetizacoes[i:i+n], beta, N), "número de passos Monte Carlo para temperatura = " + str(round(temp,2)), "Susceptibilidade Magnética",
                           "Variação da Susceptibilidade Magnética")
        temp= temp + step
    case 2: # Energia por spin
      for i in range(0,arr_energias.shape[0],n):
        plotar_grafo_linha(arr_energias[i:i+n]/N, "número de passos Monte Carlo para temperatura = " + str(round(temp,2)), "Energia por Spin",
                           "Variação da Energia por Spin")
        temp= temp + step
    case 3: # Magnetização por spin
      for i in range(0,arr_energias.shape[0],n):
        plotar_grafo_linha(arr_magnetizacoes[i:i+n]/N, "número de passos Monte Carlo para temperatura = " + str(round(temp,2)), "Magnetização por Spin",
                           "Variação da Magnetização por Spin")
        temp= temp + step

In [ ]:
def vary_temp(L, temp_inicial, limit, step, n_passos, n):
  beta = 1/temp_inicial
  N = L**2

  for temp in range(int(temp_inicial*100),int(limit*100+1),int(step*100)):
    temp = temp/100
    if (temp==temp_inicial):
        arr_energias, arr_magnetizacoes = fazer_alg_metropolis_n_vezes(n, L, temp, n_passos)
    else:
        arr_energias_aux, arr_magnetizacoes_aux = fazer_alg_metropolis_n_vezes(n, L, temp, n_passos)
        arr_energias= np.concatenate((arr_energias,arr_energias_aux),0)
        arr_magnetizacoes=np.concatenate((arr_magnetizacoes,arr_magnetizacoes_aux),0)
  return arr_energias, arr_magnetizacoes



In [ ]:
def plot_vary_temp(temp_inicial, arr_energias, arr_magnetizacoes, type):
  match tipo:
    case 0: # Calor específico
      for i in range(0,arr_energias.shape[0],n):
        plotar_grafo_linha(calc_specific_heat(arr_energias[i:i+n], beta, N), "número de passos Monte Carlo para temperatura = " + str(round(temp,2)), "Calor Específico", "Variação do Calor Específico")
        temp= temp + step
    case 1: # Susceptibilidade magnética
      for i in range(0,arr_energias.shape[0],n):
        plotar_grafo_linha(calc_magnet_susc(arr_magnetizacoes[i:i+n], beta, N), "número de passos Monte Carlo para temperatura = " + str(round(temp,2)), "Susceptibilidade Magnética",
                           "Variação da Susceptibilidade Magnética")
        temp= temp + step
    case 2: # Energia por spin
      for i in range(0,arr_energias.shape[0],n):
        plotar_grafo_linha(arr_energias[i:i+n]/N, "número de passos Monte Carlo para temperatura = " + str(round(temp,2)), "Energia por Spin",
                           "Variação da Energia por Spin")
        temp= temp + step
    case 3: # Magnetização por spin
      for i in range(0,arr_energias.shape[0],n):
        plotar_grafo_linha(arr_magnetizacoes[i:i+n]/N, "número de passos Monte Carlo para temperatura = " + str(round(temp,2)), "Magnetização por Spin",
                           "Variação da Magnetização por Spin")
        temp= temp + step

In [ ]:
# Exemplo de chamada da função para computação dos gráficos Calor Específico x Temperatura
L = 32
temp_inicial = 0.4
limit=3.0
step=0.2
n_passos = 1000
n = 5
tipo = 0
arr_energias, arr_magnetizacoes = vary_temp(L, temp_inicial, limit, step, n_passos, n)
plot_vary_temp(temp_inicial, arr_energias, arr_magnetizacoes, tipo)